In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

import ase
from ase.neighborlist import natural_cutoffs, NeighborList
from ase.io.trajectory import Trajectory
from ase.io import read as ase_read

import os
import json
import sys
sys.path.append("../py_src/")

import hydrogen_tracing
import config_plot

import re

In [ ]:
is_rutile = True

In [ ]:
rutile_dirs = [
    "rutile-001-nd-0",
    "rutile-011-nd-0",
    "rutile-100-nd-0",
    "rutile-110-nd-0",
]

anatase_dirs = [
    "anatase-100-nd-0",
    "anatase-101-nd-0", 
    "anatase-110-nd-0", 
]

if is_rutile:
    target_dirs = rutile_dirs
    savestr = 'rutile'
else:
    target_dirs = anatase_dirs
    savestr = 'anatase'

In [ ]:
all_counts = []
all_config_codes = []
all_hop_lists = []

all_bins = []
bin_maxes = []
bin_mins = []

total_min = -9
total_max = 0

for dirname in target_dirs:
    dirname = "../test_data/230302_short_traj_publike_all/" + dirname

    with open(os.path.join(dirname, os.path.basename(dirname)+'_non_numpy.json'), 'r') as f:
            non_numpy_results = json.load(f)
            config_dict = non_numpy_results['config_dict']
            special_list = non_numpy_results['special_list']
            f.close()

    counts = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_counts.txt'))
    config_codes = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_config_codes.txt'))
    hop_list = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_hop_list.txt'))

    all_counts.append(np.asarray(counts, dtype=np.int16))
    all_config_codes.append(np.asarray(config_codes, dtype=np.int8))
    all_hop_lists.append(np.asarray(hop_list, dtype=np.int8))

    # Make bin counts from last hop_list
    all_bins.append(np.bincount(all_hop_lists[-1]-np.min(all_hop_lists[-1])))
    bin_mins.append(np.min(all_hop_lists[-1]))
    bin_maxes.append(np.max(all_hop_lists[-1]))

    total_min = np.min(bin_mins)
    total_max = np.max(bin_maxes)
        
total_min = -9

In [ ]:
buff_bins = np.zeros((len(target_dirs), total_max-total_min+1))
for ii_bin, bincount in enumerate(all_bins):
    buff_bins[ii_bin, bin_mins[ii_bin]-total_min:bin_mins[ii_bin]-total_min+len(bincount)] = bincount

all_bins = buff_bins
label_range = np.arange(total_min, total_max+1)

In [ ]:
%matplotlib inline

# -9, -8, -7, -6, -5 ... -1
labels = ["Exchange of H?", "Undef.", "H20+Ti->H+HOTi\nH+HOTi->H2O+Ti", "Flyby", "Surface Hopping", "Flyby", "H20 close to HOTi", "OTi gaining H", "OTi losing H"]
labels += np.arange(0, np.max(total_max)+1).tolist()

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_title("Analysis of Ti hopping for H2O Splitting")
ax.set_xlabel('Error Codes \ Number of Hops')
ax.set_ylabel('# of occurences')

n_bars = len(target_dirs)
if n_bars % 2: # not even:
    offsets = (np.arange(1, n_bars+1) - np.ceil(n_bars/2.)) / n_bars
else:
    offsets = (np.arange(n_bars) + 0.5 - (n_bars/2)) / n_bars

for ii_dir in range(n_bars):
    hop_counts = all_bins[ii_dir]
    ax.bar(label_range + offsets[ii_dir], hop_counts, width=1./n_bars, label=target_dirs[ii_dir])

ax.set_xticks(
    label_range,
    labels,
    rotation=-45, ha="left"
)
ax.legend()
plt.show()

In [ ]:
cutoff_lim = 1
for ii_label, label in enumerate(labels):
    try:
        int(label)
        num_ind = ii_label
        break
    except:
        pass
    
plot_bins = all_bins[:, num_ind:]
plot_bins[plot_bins<=cutoff_lim] = 0

In [ ]:
print(labels)
print(plot_bins)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
plt.rcParams['hatch.linewidth'] = 2
plt.rcParams.update({'hatch.color': 'w'})

if is_rutile:
    x_range = np.arange(4)
else:
    x_range = np.arange(3)

width = 0.4

ax.bar(x=x_range-width/2, height=plot_bins[:, 0], width=width, color='tab:blue')
ax.bar(x=x_range+width/2, height=plot_bins[:, 1], width=width, color='tab:orange')
ax.bar(x=x_range-width/2, height=plot_bins[:, 2], bottom=plot_bins[:, 0], width=width, color='tab:blue', hatch='//')
try:
    ax.bar(x=x_range+width/2, height=plot_bins[:, 3], bottom=plot_bins[:, 1], width=width, color='tab:orange', hatch='//')
except: # If there is no two step dissociations
    pass 

ax.set_ylim(ax.get_ylim()[0], ax.get_ylim()[1]+50)

ax.set_xticks(x_range, target_dirs, rotation=-30, horizontalalignment='center')
if False:
    ax.legend(handles=[
        mpatches.Patch(facecolor='tab:blue', edgecolor='w', label='one-step dissociations'),
        mpatches.Patch(facecolor='tab:orange', edgecolor='w', label='two-step dissociations'),
        # mpatches.Patch(facecolor='tab:blue', edgecolor='w', label='one-step proton transfers', hatch='//'),
        mpatches.Patch(facecolor='tab:orange', edgecolor='w', label='two-step proton transfers', hatch='//')
    ])

fig.savefig("paperstep_%s.pdf"%savestr, format='pdf', bbox_inches='tight', dpi=300)
plt.show()